# Comparing different distance correlation functions

This notebook has two purposes:

1)  to compare the distance correlation results from the [dcor library](https://github.com/vnmabus/dcor) with those from [distcorr](https://gist.github.com/satra/aa3d19a12b74e9ab7941).

2) to write a [`numpy` only solution](https://stackoverflow.com/a/58084884/1034648) of [this excellent SO Answer](https://stackoverflow.com/a/29731899/1034648) on how to create a square distance matrix for a Pandas DataFrame.

#### Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
import seaborn as sns
import dcor
import copy

#### Data loading

In [2]:
data = pd.read_csv('../data/Table2_Hunt_2013_edit.csv')

In [3]:
data = data.loc[:, ['Position', 'Gross pay', 'Phi-h', 'Pressure', 'Random 1', 'Random 2', 'Gross pay transform', 'Production']]
data.describe()

,Position,Gross pay,Phi-h,Pressure,Random 1,Random 2,Gross pay transform,Production
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.885714,9.823810,68.880952,15.285714,10.190476,292.714286,16.579524,33.428571
std,0.708721,5.948521,45.167894,2.759400,6.439092,59.429069,6.543793,15.141909
min,1.000000,0.100000,0.500000,10.000000,1.000000,210.000000,3.540000,7.710000
25%,1.100000,4.900000,24.600000,14.000000,6.000000,245.000000,11.520000,22.670000
50%,2.000000,10.000000,72.900000,16.000000,10.000000,273.000000,16.900000,36.420000
75%,2.200000,15.100000,100.000000,17.000000,13.000000,340.000000,21.970000,44.200000
max,2.900000,19.100000,160.000000,20.000000,21.000000,395.000000,29.250000,59.200000


In [4]:
data.rename(index=str, columns={"Gross pay transform": "Gross pay tr"}, inplace=True)

#### Distance correlation from DCOR

Distance correlation from the [dcor library](https://github.com/vnmabus/dcor) applied using a modification from [this SO answer on Euclidean distance](https://stackoverflow.com/a/29731899/1034648) to get a square matrix

In [5]:
print ("distance correlation = {:.2f}".format(dcor.distance_correlation(data['Production'], 
                                                                        data['Phi-h'])))
print ("distance correlation = {:.2f}".format(dcor.distance_correlation(data['Production'], 
                                                                        data['Position'])))

distance correlation = 0.88
distance correlation = 0.45


#### Alternative distance correlation
`distcorr` [from here](https://gist.github.com/satra/aa3d19a12b74e9ab7941)

In [6]:
from scipy.spatial.distance import pdist, squareform
import numpy as np


def distcorr(X, Y):
    """ Compute the distance correlation function
    
    >>> a = [1,2,3,4,5]
    >>> b = np.array([1,2,9,4,4])
    >>> distcorr(a, b)
    0.762676242417
    """
    X = np.atleast_1d(X)
    Y = np.atleast_1d(Y)
    if np.prod(X.shape) == len(X):
        X = X[:, None]
    if np.prod(Y.shape) == len(Y):
        Y = Y[:, None]
    X = np.atleast_2d(X)
    Y = np.atleast_2d(Y)
    n = X.shape[0]
    if Y.shape[0] != X.shape[0]:
        raise ValueError('Number of samples must match')
    a = squareform(pdist(X))
    b = squareform(pdist(Y))
    A = a - a.mean(axis=0)[None, :] - a.mean(axis=1)[:, None] + a.mean()
    B = b - b.mean(axis=0)[None, :] - b.mean(axis=1)[:, None] + b.mean()
    
    dcov2_xy = (A * B).sum()/float(n * n)
    dcov2_xx = (A * A).sum()/float(n * n)
    dcov2_yy = (B * B).sum()/float(n * n)
    dcor = np.sqrt(dcov2_xy)/np.sqrt(np.sqrt(dcov2_xx) * np.sqrt(dcov2_yy))
    return dcor

In [7]:
print ("distance correlation = {:.2f}".format(distcorr(data['Production'], 
                                                                        data['Phi-h'])))
print ("distance correlation = {:.2f}".format(distcorr(data['Production'], 
                                                                        data['Position'])))

distance correlation = 0.88
distance correlation = 0.45


##### Same result, great!!!

##### Now let's try on the iris dataset

In [8]:
from sklearn.datasets import load_iris
iris = load_iris()
iris_df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [9]:
iris_df = iris_df.drop('target', axis=1)

In [10]:
print ("dcor distance correlation = {:.3f}".format(dcor.distance_correlation(iris_df['sepal length (cm)'], 
                                                                        iris_df['petal length (cm)'])))

dcor distance correlation = 0.859


In [11]:
print ("distcorr distance correlation = {:.3f}".format(distcorr(iris_df['sepal length (cm)'], iris_df['petal length (cm)'])))

distcorr distance correlation = 0.859


##### Same result, again!!!

#### Now making the `numpy` modification of [this SO answer](https://stackoverflow.com/a/29731899/1034648)

Testing it with both `dcor` and `pearsonr`

On the production data:

In [12]:
distcorr = lambda column1, column2: dcor.distance_correlation(column1, column2)
rslt = data.apply(lambda col1: data.apply(lambda col2: distcorr(col1, col2)))
pd.options.display.float_format = '{:,.2f}'.format
rslt

,Position,Gross pay,Phi-h,Pressure,Random 1,Random 2,Gross pay tr,Production
Position,1.00,0.28,0.32,0.22,0.32,0.54,0.23,0.45
Gross pay,0.28,1.00,0.89,0.34,0.32,0.45,0.97,0.91
Phi-h,0.32,0.89,1.00,0.27,0.35,0.54,0.83,0.88
Pressure,0.22,0.34,0.27,1.00,0.27,0.31,0.38,0.43
Random 1,0.32,0.32,0.35,0.27,1.00,0.23,0.35,0.29
Random 2,0.54,0.45,0.54,0.31,0.23,1.00,0.39,0.54
Gross pay tr,0.23,0.97,0.83,0.38,0.35,0.39,1.00,0.84
Production,0.45,0.91,0.88,0.43,0.29,0.54,0.84,1.00


In [13]:
rslt_np = np.apply_along_axis(lambda col1: np.apply_along_axis(lambda col2: distcorr(col1, col2), axis = 0, arr=data), axis =0, arr=data)
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
rslt_np

array([[1.00, 0.28, 0.32, 0.22, 0.32, 0.54, 0.23, 0.45],
       [0.28, 1.00, 0.89, 0.34, 0.32, 0.45, 0.97, 0.91],
       [0.32, 0.89, 1.00, 0.27, 0.35, 0.54, 0.83, 0.88],
       [0.22, 0.34, 0.27, 1.00, 0.27, 0.31, 0.38, 0.43],
       [0.32, 0.32, 0.35, 0.27, 1.00, 0.23, 0.35, 0.29],
       [0.54, 0.45, 0.54, 0.31, 0.23, 1.00, 0.39, 0.54],
       [0.23, 0.97, 0.83, 0.38, 0.35, 0.39, 1.00, 0.84],
       [0.45, 0.91, 0.88, 0.43, 0.29, 0.54, 0.84, 1.00]])

In [14]:
distance = lambda column1, column2: pearsonr(column1, column2)[0]
rslt = data.apply(lambda col1: data.apply(lambda col2: distance(col1, col2)))
pd.options.display.float_format = '{:,.2f}'.format
rslt

,Position,Gross pay,Phi-h,Pressure,Random 1,Random 2,Gross pay tr,Production
Position,1.00,-0.03,-0.21,-0.03,-0.02,-0.53,0.03,-0.41
Gross pay,-0.03,1.00,0.85,0.09,0.21,0.27,0.97,0.85
Phi-h,-0.21,0.85,1.00,0.03,0.25,0.39,0.81,0.86
Pressure,-0.03,0.09,0.03,1.00,-0.14,0.12,0.01,0.39
Random 1,-0.02,0.21,0.25,-0.14,1.00,-0.02,0.26,0.13
Random 2,-0.53,0.27,0.39,0.12,-0.02,1.00,0.18,0.46
Gross pay tr,0.03,0.97,0.81,0.01,0.26,0.18,1.00,0.77
Production,-0.41,0.85,0.86,0.39,0.13,0.46,0.77,1.00


In [15]:
rslt_np = np.apply_along_axis(lambda col1: np.apply_along_axis(lambda col2: pearsonr(col1, col2)[0], 
                                                               axis = 0, arr=data), 
                              axis =0, arr=data)
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
rslt_np

array([[1.00, -0.03, -0.21, -0.03, -0.02, -0.53, 0.03, -0.41],
       [-0.03, 1.00, 0.85, 0.09, 0.21, 0.27, 0.97, 0.85],
       [-0.21, 0.85, 1.00, 0.03, 0.25, 0.39, 0.81, 0.86],
       [-0.03, 0.09, 0.03, 1.00, -0.14, 0.12, 0.01, 0.39],
       [-0.02, 0.21, 0.25, -0.14, 1.00, -0.02, 0.26, 0.13],
       [-0.53, 0.27, 0.39, 0.12, -0.02, 1.00, 0.18, 0.46],
       [0.03, 0.97, 0.81, 0.01, 0.26, 0.18, 1.00, 0.77],
       [-0.41, 0.85, 0.86, 0.39, 0.13, 0.46, 0.77, 1.00]])

On the iris data:

In [16]:
distance = lambda column1, column2: pearsonr(column1, column2)[0]
rslt = iris_df.apply(lambda col1: iris_df.apply(lambda col2: distance(col1, col2)))
pd.options.display.float_format = '{:,.2f}'.format
rslt

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
sepal length (cm),1.00,-0.12,0.87,0.82
sepal width (cm),-0.12,1.00,-0.43,-0.37
petal length (cm),0.87,-0.43,1.00,0.96
petal width (cm),0.82,-0.37,0.96,1.00


In [17]:
rslt_np = np.apply_along_axis(lambda col1: np.apply_along_axis(lambda col2: pearsonr(col1, col2)[0], 
                                                               axis = 0, arr=iris_df), 
                              axis =0, arr=iris_df)
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
rslt_np

array([[1.00, -0.12, 0.87, 0.82],
       [-0.12, 1.00, -0.43, -0.37],
       [0.87, -0.43, 1.00, 0.96],
       [0.82, -0.37, 0.96, 1.00]])

In [18]:
dist_corr = lambda column1, column2: dcor.distance_correlation(column1, column2)
rslt = iris_df.apply(lambda col1: iris_df.apply(lambda col2: dist_corr(col1, col2)))
pd.options.display.float_format = '{:,.2f}'.format
rslt

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
sepal length (cm),1.00,0.31,0.86,0.83
sepal width (cm),0.31,1.00,0.54,0.51
petal length (cm),0.86,0.54,1.00,0.97
petal width (cm),0.83,0.51,0.97,1.00


In [19]:
rslt_np = np.apply_along_axis(lambda col1: np.apply_along_axis(lambda col2: dcor.distance_correlation(col1, col2), 
                                                               axis = 0, arr=iris_df), 
                              axis =0, arr=iris_df)
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
rslt_np

array([[1.00, 0.31, 0.86, 0.83],
       [0.31, 1.00, 0.54, 0.51],
       [0.86, 0.54, 1.00, 0.97],
       [0.83, 0.51, 0.97, 1.00]])

##### Does it work with nans?


In [20]:
print(iris_df.head(2))
iris_df.loc[0,2:4] =np.nan
print(iris_df.head(2))

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0               5.10              3.50               1.40              0.20
1               4.90              3.00               1.40              0.20
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0               5.10              3.50                nan               nan
1               4.90              3.00               1.40              0.20


In [21]:
rslt_np = np.apply_along_axis(lambda col1: np.apply_along_axis(lambda col2: distcorr(col1, col2), axis = 0, arr=iris_df), axis =0, arr=iris_df)
float_formatter = lambda x: "%.2f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
rslt_np

array([[1.00, 0.31, nan, nan],
       [0.31, 1.00, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]])